In [10]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

%matplotlib inline

In [11]:
# Adjust paths here, if needed
OVERLAY_PATH = "../images/PB_Maps_Overlay.png"
SEG_FMT = "mapping/mapping_{}.png"
POINTS_FMT = "mapping/anchor_points_0{}.csv"
IMG_FMT = "../../../Datasets/mapping/voc/JPEGImages/0{}.jpg"
VIS_FMT = "../../../Datasets/mapping/voc/SegmentationClassVisualization/0{}.jpg"
COLORING_FMT = "../../../Datasets/mapping/voc/SegmentationClassPNG/0{}.png"

In [12]:
overlay = cv2.imread(OVERLAY_PATH)[:, :, ::-1]
mask = np.ones_like(overlay) * 255

for n in ["1", "2", "3"]:
    print(n)
    
    seg_map = cv2.imread(SEG_FMT.format(n), cv2.IMREAD_GRAYSCALE)
    planes = np.unique(seg_map)
    
    # Just for some visualization stuff -> can be commented out, if not needed
    # img = cv2.imread(IMG_FMT.format(n))[:, :, ::-1]
    # vis = cv2.imread(VIS_FMT.format(n))[:, :, ::-1]
    # coloring = cv2.imread(COLORING_FMT.format(n))[:, :, ::-1]
    
    # read anchor points
    points = pd.read_csv(POINTS_FMT.format(n))
    
    mask_local = np.ones_like(overlay) * 255
    hms = []
    
    # calculate homography for every plane
    for plane in planes[1:]:
        
        print(plane)
        
        # img_color = vis.copy()
        enc = overlay.copy()
        
        anchors = points[points["Plane"] == plane]
        
        # plot anchors in img and enclosure
        if 0:
            for _, anchor in anchors.iterrows():
                img_anchor = [anchor["Img_x"], anchor["Img_y"]]
                enc_anchor = [anchor["Enc_x"], anchor["Enc_y"]]
                img_color = cv2.circle(img_color, img_anchor, 10, [255, 0, 0], 3)
                enc = cv2.circle(enc, enc_anchor, 10, [255, 0, 0], 3)
        else:
            # calculate homography and draw mapped anchors against gt
            img_anchors = np.array([[a["Img_x"], a["Img_y"]] for _, a in anchors.iterrows()])
            enc_anchors = np.array([[a["Enc_x"], a["Enc_y"]] for _, a in anchors.iterrows()])
            
            # calculate homography
            h, _ = cv2.findHomography(np.array(img_anchors), np.array(enc_anchors))
            hms.append(h)
            
            # transform all image anchor points
            res = np.append(img_anchors, np.ones((img_anchors.shape[0], 1)), axis=1) @ np.transpose(h)
            res = res[:, :2] / res[:, 2, np.newaxis]
            
            # calculate mean absolute error of anchor points
            print("MAE: {}".format(np.mean(np.linalg.norm(enc_anchors - res))))
            
            # draw enclosure anchors and transformed image anchors into enclosure
            for r, e in zip(res.astype(np.int32), enc_anchors):
                enc = cv2.circle(enc, e, 10, [255, 0, 0], 3)
                enc = cv2.circle(enc, r, 10, [0, 0, 255], 3)
        
        # plot the image and enclosure    
        if 0:
            fig, ax = plt.subplots(1, 1, figsize=(16, 9))
            ax.imshow(img_color)
            plt.show()
            fig, ax = plt.subplots(1, 1, figsize=(16, 9))
            ax.imshow(enc)
            plt.show()
          
    # plot transformed planes into enclosure
    if 0:
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                # ignore background
                if seg_map[i, j] != 0:                   
                    # get homography matrix
                    h = hms[seg_map[i, j] - 1]
                    
                    # transform point
                    enc_coords = h @ np.transpose(np.array([j, i, 1]))
                    enc_coords = (enc_coords[:2] / enc_coords[2]).astype(np.int32)
                    
                    # sort out problematic coordinates
                    if np.any(enc_coords < np.zeros(2)) or np.any(enc_coords >= np.array(overlay.shape[:2][::-1])):
                        continue
                    
                    # color mast
                    mask[enc_coords[1], enc_coords[0], 0] = coloring[i, j, 0]
                    mask[enc_coords[1], enc_coords[0], 1] = coloring[i, j, 1]
                    mask[enc_coords[1], enc_coords[0], 2] = coloring[i, j, 2]
                    mask_local[enc_coords[1], enc_coords[0], 0] = coloring[i, j, 0]
                    mask_local[enc_coords[1], enc_coords[0], 1] = coloring[i, j, 1]
                    mask_local[enc_coords[1], enc_coords[0], 2] = coloring[i, j, 2]
                    
        # save plots      
        mean_overlay = np.mean(overlay, axis=-1) / 255
        fig, ax = plt.subplots(1, 1, figsize=(16, 9))
        colored_overlay = (mask * mean_overlay[..., np.newaxis]).astype(np.uint8)
        colored_overlay_local = (mask_local * mean_overlay[..., np.newaxis]).astype(np.uint8)
        ax.imshow(colored_overlay)
        plt.imsave("/tmp/mapping_{}.pdf".format(n), colored_overlay)
        plt.imsave("/tmp/mapping_local_{}.pdf".format(n), colored_overlay_local)
        plt.imsave("/tmp/mapping_{}.png".format(n), colored_overlay)
        plt.imsave("/tmp/mapping_local_{}.png".format(n), colored_overlay_local)
        plt.show()
        
    # save homograhpy mapping
    if 1:
        with open("mapping/transformations_{}.pt".format(n), "wb") as f:
            pickle.dump(hms, f)
        cv2.imwrite("mapping/mapping_{}.png".format(n), seg_map)

            
    

1
1
MAE: 23.427679921234827
2
MAE: 33.581022389224856
3
MAE: 3.715316469963043e-12
4
MAE: 12.575690795406215
5
MAE: 15.276761076767011
6
MAE: 49.338101098686636
7
MAE: 3.48208512480365e-11
8
MAE: 13.572859835738107
9
MAE: 7.190186943645084e-13
10
MAE: 75.65569743162557
11
MAE: 1.1814682745140617e-12
12
MAE: 3.3339525313329386e-12
13
MAE: 54.41280604136964
14
MAE: 4.217153443958388e-12
2
1
MAE: 7.771182219636347
2
MAE: 82.43552745660082
3
MAE: 25.308772754954926
4
MAE: 10.579951686545009
5
MAE: 1.8471924715040497e-12
6
MAE: 19.49902893189564
7
MAE: 26.234931003891717
3
1
MAE: 2.990633774508275e-12
2
MAE: 17.68502565828301
3
MAE: 31.198651897768478
4
MAE: 1.5752910326854155e-12
5
MAE: 16.278000634771008
6
MAE: 22.28935345886879
7
MAE: 76.14293686808281
8
MAE: 23.791311936292864
9
MAE: 59.825715322224205
10
MAE: 2.671035900782665e-12
11
MAE: 6.215725775985027
12
MAE: 3.673334145197148e-12
13
MAE: 2.0792538734012
14
MAE: 3.811477339295427e-12
